In [3]:
# docker pull pgvector/pgvector:pg17
# docker volume create pgvector-data
# docker run --name pgvector-container -e POSTGRES_PASSWORD=rootroot -p 5433:5432 -h 127.0.0.1 -v pgvector-data:/var/lib/postgresql/data -d pgvector/pgvector:pg17

# 🔬 Lab: Exploring Vector Databases with `pgvector` and PostgreSQL

---

## 🧠 Objective
In this lab, you will:
- Understand what a vector database is and why it's used in AI/ML.
- Install and configure `pgvector` in a PostgreSQL environment.
- Connect to the PostgreSQL database using `pgAdmin`.
- Store and search vector embeddings using SQL queries.
- Use Python to interact with the database and perform similarity searches.

---

## 🛠️ Requirements
- PostgreSQL 15+
- `pgvector` extension installed
- `pgAdmin` (for GUI interaction)
- Python 3.8+
- Python packages: `psycopg2`, `sentence_transformer`

---

In [ ]:
# Activate virtual environment and install required libraries 
%%capture
!python3 -m venv .venv
!source .venv/bin/activate

!pip install psycopg2
!pip install tqdm
!pip install sentence_transformers

In [1]:
from sentence_transformers import SentenceTransformer
import psycopg2

📚 Lab Tasks

We will try to understand how vectorDB work by embedding the following corpus, and do a similarity check after connecting with vector database.

In [2]:
corpus = [
    "A man is eating food.",
    "A man is eating a piece of bread.",
    "The girl is carrying a baby.",
    "A man is riding a horse.",
    "A woman is playing violin.",
    "Two men pushed carts through the woods.",
    "A man is riding a white horse on an enclosed ground.",
    "A monkey is playing drums.",
    "A cheetah is running behind its prey.",
]

Task1: Generate Embeddings (e.g., using OpenAI API)

In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(corpus)
print(embeddings.shape)

(9, 384)


Task2: Connect to the "vectorDB"

In [5]:
conn = psycopg2.connect(
    host="localhost",
    database="vectorDB",
    port=5433,
    user="postgres",
    password="rootroot",
)

In [6]:
cur = conn.cursor()                
print("Connected Successfully")

Connected Successfully


Task 3: Enable vector extension.

Task 4: Create a Table with a Vector Column.

In [7]:
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
cur.execute("CREATE TABLE IF NOT EXISTS words (id bigserial PRIMARY KEY, embedding vector(384), sentence text);")
conn.commit()

Task 5: Insert Data into the Table

In [8]:
for emb, sen in zip(embeddings, corpus):
    cur.execute("INSERT INTO words (embedding, sentence) VALUES (%s, %s)", (emb.tolist(), sen))
conn.commit()

Task 6: Perform a Similarity Search.


In [9]:
query_emb = model.encode("The man is").tolist()

cur.execute(f"SELECT id, sentence FROM words ORDER BY embedding <-> '{query_emb}' LIMIT 3")
result = cur.fetchall()

Task 7: Print the results.


In [10]:
result

[(1, 'A man is eating food.'),
 (4, 'A man is riding a horse.'),
 (7, 'A man is riding a white horse on an enclosed ground.')]

In [11]:
conn.close()